In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os

INPUT_CSV_TRANSFORM = os.getenv("INPUT_CSV_TRANSFORM")

assert INPUT_CSV_TRANSFORM != None, "Missing INPUT_CSV_TRANSFORM"

Reading data:

In [3]:
import numpy as np
import pandas as pd

import transform_investing_pandas as ti

from common.constants import BALANCE_ARRAY_COLS, INCOME_ARRAY_COLS

%load_ext autoreload
%autoreload 1
%aimport common, transform_investing_pandas

In [4]:
raw_df = pd.read_csv(INPUT_CSV_TRANSFORM)
raw_df.head()

,stock_id,source_id,page_name,extraction_time,metric,value
0,CIB:NYSE,investing,general,2022-10-14T18:52:58.033317,currency,USD
1,CIB:NYSE,investing,general,2022-10-14T18:52:58.033317,price,25.09
2,CIB:NYSE,investing,general,2022-10-14T18:52:58.033317,shares,"961,827,000"
3,AAPL:NASDAQ,investing,balance-sheet,2022-10-14T18:52:58.374804,date_years,"2022,2022,2021,2021"
4,AAPL:NASDAQ,investing,general,2022-10-14T18:52:58.633432,currency,USD


In [5]:
raw_df.query('source_id == "google"')

,stock_id,source_id,page_name,extraction_time,metric,value
84,CIB:NYSE,google,quote,2022-10-14T18:53:01.145837,price,25
85,CIB:NYSE,google,quote,2022-10-14T18:53:01.145837,pe_ratio,4.66
86,AAPL:NASDAQ,google,quote,2022-10-14T18:53:01.299287,price,138.69
87,AAPL:NASDAQ,google,quote,2022-10-14T18:53:01.299287,pe_ratio,22.91
88,EC:NYSE,google,quote,2022-10-14T18:53:01.808976,price,9.2
89,EC:NYSE,google,quote,2022-10-14T18:53:01.808976,pe_ratio,3.21


In [6]:
investing_df = (
    raw_df
    .query("source_id == 'investing'")
    .drop(columns=["source_id"])
    .set_index(["stock_id", "page_name", "extraction_time"])
    .sort_index()
)
investing_df.head(15)

metric  \
stock_id      page_name        extraction_time                                 
AAPL:NASDAQ   balance-sheet    2022-10-14T18:52:58.374804         date_years   
                               2022-10-14T18:52:58.374804   date_days_months   
                               2022-10-14T18:52:58.374804             equity   
                               2022-10-14T18:52:58.374804        money_units   
              general          2022-10-14T18:52:58.633432           currency   
                               2022-10-14T18:52:58.633432              price   
                               2022-10-14T18:52:58.633432             shares   
              income-statement 2022-10-14T18:52:59.347448         date_years   
                               2022-10-14T18:52:59.347448   date_days_months   
                               2022-10-14T18:52:59.347448           reveneus   
                               2022-10-14T18:52:59.347448      gross_profits   
                               2022-10-14T18:52:59.347448  operating_incomes   
                               2022-10-14T18:52:59.347448        net_incomes   
                               2022-10-14T18:52:59.347448        money_units   
BCOLOMBIA:BVC balance-sheet    2022-10-14T18:53:00.532916         date_years   

                                                                                                       value  
stock_id      page_name        extraction_time                                                                
AAPL:NASDAQ   balance-sheet    2022-10-14T18:52:58.374804                                2022,2022,2021,2021  
                               2022-10-14T18:52:58.374804                            25/06,26/03,25/12,25/09  
                               2022-10-14T18:52:58.374804                            58107,67399,71932,63090  
                               2022-10-14T18:52:58.374804  * In Millions of USD (except for per share items)  
              general          2022-10-14T18:52:58.633432                                                USD  
                               2022-10-14T18:52:58.633432                                             138.52  
                               2022-10-14T18:52:58.633432                                     16,070,752,000  
              income-statement 2022-10-14T18:52:59.347448                                2022,2022,2021,2021  
                               2022-10-14T18:52:59.347448                            25/06,26/03,25/12,25/09  
                               2022-10-14T18:52:59.347448                           82959,97278,123945,83360  
                               2022-10-14T18:52:59.347448                            35885,42559,54243,35174  
                               2022-10-14T18:52:59.347448                            23076,29979,41488,23786  
                               2022-10-14T18:52:59.347448                            19442,25010,34630,20551  
                               2022-10-14T18:52:59.347448  * In Millions of USD (except for per share items)  
BCOLOMBIA:BVC balance-sheet    2022-10-14T18:53:00.532916                                2022,2022,2021,2021

In [7]:
investing_df.reset_index()['page_name'].unique()

array(['balance-sheet', 'general', 'income-statement'], dtype=object)

## Wrangling pages

Page: `general`

In [8]:
general_df = ti.get_pivoted_page_df(investing_df, "general")
general_df

,,metric,currency,price,shares
stock_id,page_name,extraction_time,,,
AAPL:NASDAQ,general,2022-10-14T18:52:58.633432,USD,138.52,"16,070,752,000"
BCOLOMBIA:BVC,general,2022-10-14T18:52:59.830715,COP,"34,200.0","961,827,000"
CIB:NYSE,general,2022-10-14T18:52:58.033317,USD,25.09,"961,827,000"
EC:NYSE,general,2022-10-14T18:52:58.861408,USD,9.22,"41,116,694,690"
ECOPETROL:BVC,general,2022-10-14T18:52:59.144838,COP,"2,163.0","41,116,694,690"
PFBCOLOM:BVC,general,2022-10-14T18:53:00.307557,COP,"28,560.0","961,827,000"


In [9]:
general_df = ti.transform_general_page(general_df)
general_df

,,metric,currency,price,shares
stock_id,page_name,extraction_time,,,
AAPL:NASDAQ,general,2022-10-14T18:52:58.633432,USD,138.52,16070752000
BCOLOMBIA:BVC,general,2022-10-14T18:52:59.830715,COP,34200.0,961827000
CIB:NYSE,general,2022-10-14T18:52:58.033317,USD,25.09,961827000
EC:NYSE,general,2022-10-14T18:52:58.861408,USD,9.22,41116694690
ECOPETROL:BVC,general,2022-10-14T18:52:59.144838,COP,2163.0,41116694690
PFBCOLOM:BVC,general,2022-10-14T18:53:00.307557,COP,28560.0,961827000


### Page: `balance`

In [10]:
balance_df = ti.get_pivoted_page_df(investing_df, "balance-sheet")
balance_df

,,metric,date_days_months,date_years,equity,money_units
stock_id,page_name,extraction_time,,,,
AAPL:NASDAQ,balance-sheet,2022-10-14T18:52:58.374804,"25/06,26/03,25/12,25/09","2022,2022,2021,2021","58107,67399,71932,63090",* In Millions of USD (except for per share items)
BCOLOMBIA:BVC,balance-sheet,2022-10-14T18:53:00.532916,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","35099020,30200262,32234347,30261139",* In Millions of COP (except for per share items)
CIB:NYSE,balance-sheet,2022-10-14T18:52:58.783524,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","35099020,30200262,32234347,30261139",* In Millions of COP (except for per share items)
EC:NYSE,balance-sheet,2022-10-14T18:52:59.422797,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,89144201,71733024,62994784",* In Millions of (except for per share items)
ECOPETROL:BVC,balance-sheet,2022-10-14T18:52:59.614770,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,89144201,71733024,62994784",* In Millions of COP (except for per share items)
PFBCOLOM:BVC,balance-sheet,2022-10-14T18:53:00.623502,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","35099020,30200262,32234347,30261139",* In Millions of COP (except for per share items)


In [11]:
balance_df = ti.transform_money_units(balance_df)
balance_df

,,,date_days_months,date_years,equity,factor,currency
stock_id,page_name,extraction_time,,,,,
AAPL:NASDAQ,balance-sheet,2022-10-14T18:52:58.374804,"25/06,26/03,25/12,25/09","2022,2022,2021,2021","58107,67399,71932,63090",Millions,USD
BCOLOMBIA:BVC,balance-sheet,2022-10-14T18:53:00.532916,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","35099020,30200262,32234347,30261139",Millions,COP
CIB:NYSE,balance-sheet,2022-10-14T18:52:58.783524,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","35099020,30200262,32234347,30261139",Millions,COP
EC:NYSE,balance-sheet,2022-10-14T18:52:59.422797,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,89144201,71733024,62994784",NaN,NaN
ECOPETROL:BVC,balance-sheet,2022-10-14T18:52:59.614770,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,89144201,71733024,62994784",Millions,COP
PFBCOLOM:BVC,balance-sheet,2022-10-14T18:53:00.623502,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","35099020,30200262,32234347,30261139",Millions,COP


In [12]:
balance_df = ti.transform_from_arrays(balance_df, BALANCE_ARRAY_COLS)
balance_df

date_years  \
stock_id      page_name     extraction_time                         
AAPL:NASDAQ   balance-sheet 2022-10-14T18:52:58.374804       2022   
                            2022-10-14T18:52:58.374804       2022   
                            2022-10-14T18:52:58.374804       2021   
                            2022-10-14T18:52:58.374804       2021   
BCOLOMBIA:BVC balance-sheet 2022-10-14T18:53:00.532916       2022   
                            2022-10-14T18:53:00.532916       2022   
                            2022-10-14T18:53:00.532916       2021   
                            2022-10-14T18:53:00.532916       2021   
CIB:NYSE      balance-sheet 2022-10-14T18:52:58.783524       2022   
                            2022-10-14T18:52:58.783524       2022   
                            2022-10-14T18:52:58.783524       2021   
                            2022-10-14T18:52:58.783524       2021   
EC:NYSE       balance-sheet 2022-10-14T18:52:59.422797       2022   
                            2022-10-14T18:52:59.422797       2022   
                            2022-10-14T18:52:59.422797       2021   
                            2022-10-14T18:52:59.422797       2021   
ECOPETROL:BVC balance-sheet 2022-10-14T18:52:59.614770       2022   
                            2022-10-14T18:52:59.614770       2022   
                            2022-10-14T18:52:59.614770       2021   
                            2022-10-14T18:52:59.614770       2021   
PFBCOLOM:BVC  balance-sheet 2022-10-14T18:53:00.623502       2022   
                            2022-10-14T18:53:00.623502       2022   
                            2022-10-14T18:53:00.623502       2021   
                            2022-10-14T18:53:00.623502       2021   

                                                       date_days_months  \
stock_id      page_name     extraction_time                               
AAPL:NASDAQ   balance-sheet 2022-10-14T18:52:58.374804            25/06   
                            2022-10-14T18:52:58.374804            26/03   
                            2022-10-14T18:52:58.374804            25/12   
                            2022-10-14T18:52:58.374804            25/09   
BCOLOMBIA:BVC balance-sheet 2022-10-14T18:53:00.532916            30/06   
                            2022-10-14T18:53:00.532916            31/03   
                            2022-10-14T18:53:00.532916            31/12   
                            2022-10-14T18:53:00.532916            30/09   
CIB:NYSE      balance-sheet 2022-10-14T18:52:58.783524            30/06   
                            2022-10-14T18:52:58.783524            31/03   
                            2022-10-14T18:52:58.783524            31/12   
                            2022-10-14T18:52:58.783524            30/09   
EC:NYSE       balance-sheet 2022-10-14T18:52:59.422797            30/06   
                            2022-10-14T18:52:59.422797            31/03   
                            2022-10-14T18:52:59.422797            31/12   
                            2022-10-14T18:52:59.422797            30/09   
ECOPETROL:BVC balance-sheet 2022-10-14T18:52:59.614770            30/06   
                            2022-10-14T18:52:59.614770            31/03   
                            2022-10-14T18:52:59.614770            31/12   
                            2022-10-14T18:52:59.614770            30/09   
PFBCOLOM:BVC  balance-sheet 2022-10-14T18:53:00.623502            30/06   
                            2022-10-14T18:53:00.623502            31/03   
                            2022-10-14T18:53:00.623502            31/12   
                            2022-10-14T18:53:00.623502            30/09   

                                                          equity    factor  \
stock_id      page_name     extraction_time                                  
AAPL:NASDAQ   balance-sheet 2022-10-14T18:52:58.374804     58107  Millions   
                            2022-10-14T18:52:58.374804     67399  Millions 

### Page: `income`

In [13]:
income_df = ti.get_pivoted_page_df(investing_df, "income-statement")
income_df

,,metric,date_days_months,date_years,gross_profits,money_units,net_incomes,operating_incomes,reveneus
stock_id,page_name,extraction_time,,,,,,,
AAPL:NASDAQ,income-statement,2022-10-14T18:52:59.347448,"25/06,26/03,25/12,25/09","2022,2022,2021,2021","35885,42559,54243,35174",* In Millions of USD (except for per share items),"19442,25010,34630,20551","23076,29979,41488,23786","82959,97278,123945,83360"
BCOLOMBIA:BVC,income-statement,2022-10-14T18:53:00.691773,"30/06,31/03,31/12,30/09","2022,2022,2021,2021",NaN,* In Millions of COP (except for per share items),"1779695,1731858,1444744,942620",NaN,NaN
CIB:NYSE,income-statement,2022-10-14T18:52:59.489989,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","4911324,-,-,-",* In Millions of COP (except for per share items),"1779695,1731858,1444744,942620","2571281,-,-,-","5088064,-,-,-"
EC:NYSE,income-statement,2022-10-14T18:52:59.554673,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,14736268,12628781,8826762",* In Millions of (except for per share items),"-,6572640,6077214,3807248","-,12664307,10388494,7102309","-,32472744,31761115,23332762"
ECOPETROL:BVC,income-statement,2022-10-14T18:53:00.068412,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,14736268,12628781,8826762",* In Millions of COP (except for per share items),"-,6572640,6077214,3807248","-,12664307,10388494,7102309","-,32472744,31761115,23332762"
PFBCOLOM:BVC,income-statement,2022-10-14T18:53:00.462319,"30/06,31/03,31/12,30/09","2022,2022,2021,2021",NaN,* In Millions of COP (except for per share items),"1779695,1731858,1444744,942620",NaN,NaN


In [14]:
income_df = ti.transform_money_units(income_df)
income_df

,,,date_days_months,date_years,gross_profits,net_incomes,operating_incomes,reveneus,factor,currency
stock_id,page_name,extraction_time,,,,,,,,
AAPL:NASDAQ,income-statement,2022-10-14T18:52:59.347448,"25/06,26/03,25/12,25/09","2022,2022,2021,2021","35885,42559,54243,35174","19442,25010,34630,20551","23076,29979,41488,23786","82959,97278,123945,83360",Millions,USD
BCOLOMBIA:BVC,income-statement,2022-10-14T18:53:00.691773,"30/06,31/03,31/12,30/09","2022,2022,2021,2021",NaN,"1779695,1731858,1444744,942620",NaN,NaN,Millions,COP
CIB:NYSE,income-statement,2022-10-14T18:52:59.489989,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","4911324,-,-,-","1779695,1731858,1444744,942620","2571281,-,-,-","5088064,-,-,-",Millions,COP
EC:NYSE,income-statement,2022-10-14T18:52:59.554673,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,14736268,12628781,8826762","-,6572640,6077214,3807248","-,12664307,10388494,7102309","-,32472744,31761115,23332762",NaN,NaN
ECOPETROL:BVC,income-statement,2022-10-14T18:53:00.068412,"30/06,31/03,31/12,30/09","2022,2022,2021,2021","-,14736268,12628781,8826762","-,6572640,6077214,3807248","-,12664307,10388494,7102309","-,32472744,31761115,23332762",Millions,COP
PFBCOLOM:BVC,income-statement,2022-10-14T18:53:00.462319,"30/06,31/03,31/12,30/09","2022,2022,2021,2021",NaN,"1779695,1731858,1444744,942620",NaN,NaN,Millions,COP


In [15]:
income_df = ti.transform_from_arrays(income_df, INCOME_ARRAY_COLS)
income_df

date_years  \
stock_id      page_name        extraction_time                         
AAPL:NASDAQ   income-statement 2022-10-14T18:52:59.347448       2022   
                               2022-10-14T18:52:59.347448       2022   
                               2022-10-14T18:52:59.347448       2021   
                               2022-10-14T18:52:59.347448       2021   
BCOLOMBIA:BVC income-statement 2022-10-14T18:53:00.691773       2022   
                               2022-10-14T18:53:00.691773       2022   
                               2022-10-14T18:53:00.691773       2021   
                               2022-10-14T18:53:00.691773       2021   
CIB:NYSE      income-statement 2022-10-14T18:52:59.489989       2022   
                               2022-10-14T18:52:59.489989       2022   
                               2022-10-14T18:52:59.489989       2021   
                               2022-10-14T18:52:59.489989       2021   
EC:NYSE       income-statement 2022-10-14T18:52:59.554673       2022   
                               2022-10-14T18:52:59.554673       2022   
                               2022-10-14T18:52:59.554673       2021   
                               2022-10-14T18:52:59.554673       2021   
ECOPETROL:BVC income-statement 2022-10-14T18:53:00.068412       2022   
                               2022-10-14T18:53:00.068412       2022   
                               2022-10-14T18:53:00.068412       2021   
                               2022-10-14T18:53:00.068412       2021   
PFBCOLOM:BVC  income-statement 2022-10-14T18:53:00.462319       2022   
                               2022-10-14T18:53:00.462319       2022   
                               2022-10-14T18:53:00.462319       2021   
                               2022-10-14T18:53:00.462319       2021   

                                                          date_days_months  \
stock_id      page_name        extraction_time                               
AAPL:NASDAQ   income-statement 2022-10-14T18:52:59.347448            25/06   
                               2022-10-14T18:52:59.347448            26/03   
                               2022-10-14T18:52:59.347448            25/12   
                               2022-10-14T18:52:59.347448            25/09   
BCOLOMBIA:BVC income-statement 2022-10-14T18:53:00.691773            30/06   
                               2022-10-14T18:53:00.691773            31/03   
                               2022-10-14T18:53:00.691773            31/12   
                               2022-10-14T18:53:00.691773            30/09   
CIB:NYSE      income-statement 2022-10-14T18:52:59.489989            30/06   
                               2022-10-14T18:52:59.489989            31/03   
                               2022-10-14T18:52:59.489989            31/12   
                               2022-10-14T18:52:59.489989            30/09   
EC:NYSE       income-statement 2022-10-14T18:52:59.554673            30/06   
                               2022-10-14T18:52:59.554673            31/03   
                               2022-10-14T18:52:59.554673            31/12   
                               2022-10-14T18:52:59.554673            30/09   
ECOPETROL:BVC income-statement 2022-10-14T18:53:00.068412            30/06   
                               2022-10-14T18:53:00.068412            31/03   
                               2022-10-14T18:53:00.068412            31/12   
                               2022-10-14T18:53:00.068412            30/09   
PFBCOLOM:BVC  income-statement 2022-10-14T18:53:00.462319            30/06   
                               2022-10-14T18:53:00.462319            31/03   
                               2022-10-14T18:53:00.462319            31/12   
                               2022-10-14T18:53:00.462319            30/09   

                                                          gross_profits  \
stock_id      page_name        extraction_time                            
AAPL:N

In [16]:
income_df = ti.transform_date(income_df)
income_df

gross_profits  \
stock_id      page_name        extraction_time                            
AAPL:NASDAQ   income-statement 2022-10-14T18:52:59.347448         35885   
                               2022-10-14T18:52:59.347448         42559   
                               2022-10-14T18:52:59.347448         54243   
                               2022-10-14T18:52:59.347448         35174   
BCOLOMBIA:BVC income-statement 2022-10-14T18:53:00.691773           NaN   
                               2022-10-14T18:53:00.691773           NaN   
                               2022-10-14T18:53:00.691773           NaN   
                               2022-10-14T18:53:00.691773           NaN   
CIB:NYSE      income-statement 2022-10-14T18:52:59.489989       4911324   
                               2022-10-14T18:52:59.489989           NaN   
                               2022-10-14T18:52:59.489989           NaN   
                               2022-10-14T18:52:59.489989           NaN   
EC:NYSE       income-statement 2022-10-14T18:52:59.554673           NaN   
                               2022-10-14T18:52:59.554673      14736268   
                               2022-10-14T18:52:59.554673      12628781   
                               2022-10-14T18:52:59.554673       8826762   
ECOPETROL:BVC income-statement 2022-10-14T18:53:00.068412           NaN   
                               2022-10-14T18:53:00.068412      14736268   
                               2022-10-14T18:53:00.068412      12628781   
                               2022-10-14T18:53:00.068412       8826762   
PFBCOLOM:BVC  income-statement 2022-10-14T18:53:00.462319           NaN   
                               2022-10-14T18:53:00.462319           NaN   
                               2022-10-14T18:53:00.462319           NaN   
                               2022-10-14T18:53:00.462319           NaN   

                                                          net_incomes  \
stock_id      page_name        extraction_time                          
AAPL:NASDAQ   income-statement 2022-10-14T18:52:59.347448       19442   
                               2022-10-14T18:52:59.347448       25010   
                               2022-10-14T18:52:59.347448       34630   
                               2022-10-14T18:52:59.347448       20551   
BCOLOMBIA:BVC income-statement 2022-10-14T18:53:00.691773     1779695   
                               2022-10-14T18:53:00.691773     1731858   
                               2022-10-14T18:53:00.691773     1444744   
                               2022-10-14T18:53:00.691773      942620   
CIB:NYSE      income-statement 2022-10-14T18:52:59.489989     1779695   
                               2022-10-14T18:52:59.489989     1731858   
                               2022-10-14T18:52:59.489989     1444744   
                               2022-10-14T18:52:59.489989      942620   
EC:NYSE       income-statement 2022-10-14T18:52:59.554673         NaN   
                               2022-10-14T18:52:59.554673     6572640   
                               2022-10-14T18:52:59.554673     6077214   
                               2022-10-14T18:52:59.554673     3807248   
ECOPETROL:BVC income-statement 2022-10-14T18:53:00.068412         NaN   
                               2022-10-14T18:53:00.068412     6572640   
                               2022-10-14T18:53:00.068412     6077214   
                               2022-10-14T18:53:00.068412     3807248   
PFBCOLOM:BVC  income-statement 2022-10-14T18:53:00.462319     1779695   
                               2022-10-14T18:53:00.462319     1731858   
                               2022-10-14T18:53:00.462319     1444744   
                               2022-10-14T18:53:00.462319      942620   

                                                          operating_incomes  \
stock_id      page_name        extraction_time                                
AAPL:NASDAQ   income-statement 2022-10-14T18:52:59

In [17]:
income_df.shape

(24, 7)